In [ ]:
!pip install langchain openai google-search-results unstructured chromadb pinecone-client youtube-transcript-api pytube faiss-cpu langchain-community jq tiktoken tdqm python-dotenv

In [1]:
from google.colab import drive
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/Colab Notebooks/Intro_to_AI_final/data.json'

Mounted at /content/drive


In [4]:
import openai
import json
import time
import random
from google.colab import drive
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import JSONLoader
from langchain.schema import Document, HumanMessage
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from googlesearch import search
from tqdm import tqdm

In [28]:
import os
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

In [29]:
faiss_path = "/content/drive/MyDrive/Colab Notebooks/Intro_to_AI_final/faiss_index"

In [7]:
# 讀取 JSON 文件内容並打印以進行調試
with open(data_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

print("加载 JSON 文件内容:")
for i in range(5):
    item = data[i]
    print(f"文檔 {i+1}")
    print(item)

# 定義一個函數來將每個文檔轉換成文本字符串
def convert_to_text(doc):
    fields = ["課名", "教授", "修課年度", "教了什麼", "上課方式", "考試作業", "其他", "最後想說的話"]
    text_content = "\n".join([f"{field}: {doc[field]}" for field in fields if field in doc])
    return Document(page_content=text_content)

# 處理嵌套的 JSON 數據，將其轉換為文檔
documents = []
for sublist in data:
    for item in sublist:
        documents.append(convert_to_text(item))

print("轉換後的文檔:")
for i in range(5):
    doc = documents[i]
    print(f"文檔 {i+1}:")
    print(doc.page_content)

# 創建 embeddings
embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key)

加载 JSON 文件内容:
文檔 1
[{'課名': '混合信號積體電路導論', '教授': '陳佳宏', '修課年度': '110學年第2學期', '教了什麼': '雖然課名寫混合信號，實際上只會教混合信號中的ADC和DAC，而老師的專長是ADC。會將ADC的主要架構和它們的tradeoff講解一遍，然後介紹ADC的spec的定義，最後會比較仔細講解switched-capacitor circuits和comparator的電路。', '上課方式': '用有點亂的自編講義上課，有幾本參考用的課本，學校有買授權可以下載。', '考試作業': '一次期末考，open computer(疫情期間)，題目不多，有複習的話分數不難拿。共有五次作業加上期末專題。前四次是Matlab和Simulink的練習，做理想的ADC量quantization error和FFT之類的，第五次和期末專題要用Virtuoso的理想元件兜macro model做一個pipeline ADC。作業都會提供簡單的範例或教學，但感覺這些範例的來源不一，沒有整理過，通常要花一些時間摸索才能看懂。', '其他': '不點名。會用到AIC和訊號與系統的基礎知識，但不至於講得很深。', '最後想說的話': '好像號稱小DCIC，但總覺得課程安排有點隨興，好像老師想到什麼就教什麼，而且或許是怕講太多大家吸收不了，老師講話常常很繞，但又沒講新東西，很容易聽到分心。不過老師人很好，可以為了你再多繞幾次XDD作業和考試題目偶爾有些小bug，當下會蠻煩躁的，但最後還算有學到東西。有點可惜沒有講老師最擅長的delta-sigma ADC(今年好像有教了)。修課距今隔了一年，加上當時又還有疫情，所以很多東西可能不一樣了，僅供參考~--'}]
文檔 2
[{'課名': '計算機結構', '教授': '劉志尉', '修課年度': '111-2', '教了什麼': '介紹計算機結構的基本概念，沒有基礎也可以修         建議要有課本，作業跟考試都是從裡面出題', '上課方式': '用老師自製的投影片，整學期只有點一次名，他原本說有點到的額外加分         但他好像忘了(?)         去年修的時候太多人，所以規定兩個人一組         沒分到助教會幫你分', '考試作業': '四次小考10% 跟課本例題

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
# 定義一個函數來拆分文檔
def split_documents(documents, max_tokens):
    batches = []
    current_batch = []
    current_batch_size = 0
    for doc in documents:
        doc_size = len(doc.page_content.split())  # 使用 page_content 字段
        if current_batch_size + doc_size > max_tokens:
            batches.append(current_batch)
            current_batch = [doc]
            current_batch_size = doc_size
        else:
            current_batch.append(doc)
            current_batch_size += doc_size
    if current_batch:
        batches.append(current_batch)
    return batches

# 將文檔拆分為較小的部分
batches = split_documents(documents, max_tokens=20000)  # 調整 max_tokens

# 分批創建向量存儲並合併
vectorstore = None
for batch in tqdm(batches, desc="VectorStore"):
    try:
        batch_vectorstore = FAISS.from_documents(batch, embeddings)
        if vectorstore is None:
            vectorstore = batch_vectorstore
        else:
            vectorstore.merge_from(batch_vectorstore)
        time.sleep(2)  # 每個請求間添加延遲
    except openai.error.RateLimitError:
        print("Rate limit exceeded, waiting for 60 seconds")
        time.sleep(60)  # 超出速率限制時，等待更長的時間
        batch_vectorstore = FAISS.from_documents(batch, embeddings)
        if vectorstore is None:
            vectorstore = batch_vectorstore
        else:
            vectorstore.merge_from(batch_vectorstore)

# 保存向量存儲到文件
vectorstore.save_local(faiss_path)

In [30]:
vectorstore = FAISS.load_local(faiss_path, embeddings, allow_dangerous_deserialization=True)

retrieval_qa_prompt = """
請用繁體中文回答，回答盡量簡短不超過5句話。
Context:
{context}

Question: {question}

Answer:
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=retrieval_qa_prompt
)

# 使用 OpenAI API 和提示模板
llm = ChatOpenAI(openai_api_base="https://api.openai.com/v1", api_key=openai.api_key, model='gpt-4o')

# 全文搜索函數
def full_text_search(query, documents):
    results = []
    for doc in documents:
        if query in doc.page_content:
            results.append(doc)
    return results

# 查詢函數，僅使用全文搜索
def query_full_text_system(query):
    # 使用全文搜索
    text_docs = full_text_search(query, documents)
    context = "\n\n".join(text_docs)

    # 使用 LLM 生成回答
    formatted_prompt = prompt.format(context=context, question=query)
    messages = [HumanMessage(content=formatted_prompt)]
    response = llm(messages=messages)
    return response.content

# 查詢函數，結合全文搜索和向量檢索
def query_rag_system(query):
    # 從向量存儲中檢索相關文檔
    vector_docs = vectorstore.similarity_search(query)

    # 使用全文搜索來補充
    text_docs = full_text_search(query, documents)

    # 合併兩種搜索形式，去除權重
    combined_docs = {doc.page_content: doc for doc in vector_docs + text_docs}.values()
    context = "\n\n".join([doc.page_content for doc in combined_docs])

    # 使用 LLM 生成回答
    formatted_prompt = prompt.format(context=context, question=query)
    messages = [HumanMessage(content=formatted_prompt)]
    response = llm(messages=messages)

    return response.content

# 查詢測試
query = "嚴力行教授教哪堂課?"
response = query_rag_system(query)
print(response)

嚴力行教授教「現代東亞政治」課。


In [24]:
questions_path = "/content/drive/MyDrive/Colab Notebooks/Intro_to_AI_final/questions.json"
output_path = "/content/drive/MyDrive/Colab Notebooks/Intro_to_AI_final/output_FULL.json"
with open(questions_path, 'r', encoding='utf-8') as f:
    questions = json.load(f)

random.shuffle(questions)
print(questions)

# 迭代每個問題，並生成問答 pair
qa_pairs = []
for question in tqdm(questions, desc="Answering Questions"):
    answer = query_full_text_system(question)
    qa_pairs.append({"question": question, "answer": answer})

# 將問答 pair 保存為 JSON 文件
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(qa_pairs, f, ensure_ascii=False, indent=4)

print(f"\n保存到 {output_path}")

['有沒有不用寫心得的通識課?', '電子物理系(電物系)要修哪些課程?', '人社系有哪些推薦的課程?', '混合信號積體電路導論課程的考試形式是什麼?', '我是管科系大二，有哪些推薦課程?', '荊宇泰教授常當人嗎?', '我缺外語學分，有推薦的課嗎?', '推薦哪些藝術相關的課程?', '有沒有不用小組報告的通識課?', '謝秉鈞教授的機率硬嗎?', '哪些課程的教授上課會有板書?', '我想修很涼的通識課', '嚴力行教授有開哪些課?', '有哪些課程是不用考試的?', '哪些課程的期中考超過一次?', '推薦工學院的學生修生物醫學概論嗎?', '上複變函數的課需要什麼基礎知識?', '有哪些雷課不要修?', '資財系最硬的課是哪堂?', '有哪些推薦的英文課?', '哪些課程的評價偏高?', '哪個微積分教授是大刀?', '哪些課有要求要上去報paper', '我還缺一些外語學分，推薦修哪些課程?', '有推薦的資工系大三選修課嗎?', '數值軟體開發的作業多嗎?', '有推薦的通識課-領域知識嗎?', '有推薦的通識課嗎?', '哪些課會調分?', '哪些課對出席率有要求?', '高榮鴻教授教授有開哪些課?', '有沒有給分很甜的通識課?', '軍訓課主要在教些什麼?', '比較網路程式設計概論的教授們。', '我想要修很涼的課，推薦我一些課程的名字?', '有沒有推薦的微積分教授?', '哪些課程的平均分數是高於80分?', '哪位教授有開計算機結構?', '哪些課是全英文授課?', '有推薦的通識課-基本素養嗎?', '賽局理論及應用課程有幾次程式作業和小考?', '影像處理課程的考試形式是什麼?', '機械系有哪些必修?', '哪些課程的考試看考古題就夠?', '有哪些的教授是著名的大刀?', '有哪些課是大學生難度的研究所課?', '有哪些課是電機系必修?', '計算機結構課程有幾次小考和作業?', '哪些課程的課堂內容跟作業考試內容相差很大?', '哪些教授的評分標準很玄?']


Answering Questions: 100%|██████████| 50/50 [01:02<00:00,  1.24s/it]


保存到 /content/drive/MyDrive/Colab Notebooks/Intro_to_AI_final/output_FULL.json


In [22]:
questions_path = "/content/drive/MyDrive/Colab Notebooks/Intro_to_AI_final/questions.json"
output_path = "/content/drive/MyDrive/Colab Notebooks/Intro_to_AI_final/output_RAG.json"
with open(questions_path, 'r', encoding='utf-8') as f:
    questions = json.load(f)

random.shuffle(questions)
print(questions)

# 迭代每個問題，並生成問答 pair
qa_pairs = []
for question in tqdm(questions, desc="Answering Questions"):
    answer = query_rag_system(question)
    qa_pairs.append({"question": question, "answer": answer})

# 將問答 pair 保存為 JSON 文件
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(qa_pairs, f, ensure_ascii=False, indent=4)

print(f"\n保存到 {output_path}")

['謝秉鈞教授的機率硬嗎?', '哪些課程的評價偏高?', '哪些課程的平均分數是高於80分?', '賽局理論及應用課程有幾次程式作業和小考?', '混合信號積體電路導論課程的考試形式是什麼?', '嚴力行教授有開哪些課?', '有哪些課程是不用考試的?', '推薦哪些藝術相關的課程?', '哪些課程的期中考超過一次?', '高榮鴻教授教授有開哪些課?', '哪些課程的課堂內容跟作業考試內容相差很大?', '荊宇泰教授常當人嗎?', '哪些教授的評分標準很玄?', '機械系有哪些必修?', '有推薦的通識課-基本素養嗎?', '哪些課會調分?', '哪些課是全英文授課?', '有推薦的資工系大三選修課嗎?', '哪位教授有開計算機結構?', '計算機結構課程有幾次小考和作業?', '哪些課對出席率有要求?', '人社系有哪些推薦的課程?', '有哪些課是大學生難度的研究所課?', '我還缺一些外語學分，推薦修哪些課程?', '我是管科系大二，有哪些推薦課程?', '有哪些推薦的英文課?', '有推薦的通識課-領域知識嗎?', '有沒有不用寫心得的通識課?', '推薦工學院的學生修生物醫學概論嗎?', '影像處理課程的考試形式是什麼?', '有哪些課是電機系必修?', '電子物理系(電物系)要修哪些課程?', '資財系最硬的課是哪堂?', '數值軟體開發的作業多嗎?', '比較網路程式設計概論的教授們。', '有哪些的教授是著名的大刀?', '我想要修很涼的課，推薦我一些課程的名字?', '哪些課程的教授上課會有板書?', '有沒有推薦的微積分教授?', '有哪些雷課不要修?', '有推薦的通識課嗎?', '我缺外語學分，有推薦的課嗎?', '有沒有不用小組報告的通識課?', '哪些課有要求要上去報paper', '上複變函數的課需要什麼基礎知識?', '軍訓課主要在教些什麼?', '我想修很涼的通識課', '哪些課程的考試看考古題就夠?', '有沒有給分很甜的通識課?', '哪個微積分教授是大刀?']


Answering Questions: 100%|██████████| 50/50 [02:40<00:00,  3.22s/it]


保存到 /content/drive/MyDrive/Colab Notebooks/Intro_to_AI_final/output_RAG.json


In [29]:
questions_path = "/content/drive/MyDrive/Colab Notebooks/Intro_to_AI_final/questions.json"
def seletect_query(flag, path, iter):
    with open(questions_path, 'r', encoding='utf-8') as f:
        questions = json.load(f)

    for i in range(iter):
        dest_path = path[:-5] + f"_{i}.json"
        print(f"Destination: {dest_path}")
        random.shuffle(questions)
        #print(questions)

        # 迭代每個問題，並生成問答 pair
        qa_pairs = []
        if flag == "RAG":
            for question in tqdm(questions, desc="Answering Questions"):
                answer = query_rag_system(question)
                qa_pairs.append({"question": question, "answer": answer})
        elif flag == "FULL":
            for question in tqdm(questions, desc="Answering Questions"):
                answer = query_full_text_system(question)
                qa_pairs.append({"question": question, "answer": answer})

        # 將問答 pair 保存為 JSON 文件
        with open(dest_path, 'w', encoding='utf-8') as f:
            json.dump(qa_pairs, f, ensure_ascii=False, indent=4)

        print(f"保存到 {dest_path}\n")


seletect_query("RAG", "/content/drive/MyDrive/Colab Notebooks/Intro_to_AI_final/Answer/answerRAG.json", 5)
seletect_query("FULL", "/content/drive/MyDrive/Colab Notebooks/Intro_to_AI_final/Answer/answerFULL.json", 5)

Destination: /content/drive/MyDrive/Colab Notebooks/Intro_to_AI_final/Answer/answerRAG_0.json


Answering Questions: 100%|██████████| 50/50 [02:21<00:00,  2.84s/it]


保存到 /content/drive/MyDrive/Colab Notebooks/Intro_to_AI_final/Answer/answerRAG_0.json

Destination: /content/drive/MyDrive/Colab Notebooks/Intro_to_AI_final/Answer/answerRAG_1.json


Answering Questions: 100%|██████████| 50/50 [03:16<00:00,  3.94s/it]


保存到 /content/drive/MyDrive/Colab Notebooks/Intro_to_AI_final/Answer/answerRAG_1.json

Destination: /content/drive/MyDrive/Colab Notebooks/Intro_to_AI_final/Answer/answerRAG_2.json


...

保存到 /content/drive/MyDrive/Colab Notebooks/Intro_to_AI_final/Answer/answerFULL_2.json

Destination: /content/drive/MyDrive/Colab Notebooks/Intro_to_AI_final/Answer/answerFULL_3.json


Answering Questions: 100%|██████████| 50/50 [01:00<00:00,  1.20s/it]


保存到 /content/drive/MyDrive/Colab Notebooks/Intro_to_AI_final/Answer/answerFULL_3.json

Destination: /content/drive/MyDrive/Colab Notebooks/Intro_to_AI_final/Answer/answerFULL_4.json


Answering Questions: 100%|██████████| 50/50 [00:55<00:00,  1.12s/it]

保存到 /content/drive/MyDrive/Colab Notebooks/Intro_to_AI_final/Answer/answerFULL_4.json



In [33]:
from collections import Counter
result_FULL = """FULL-0
否半半半否否否否否否否否否否是否否否否否否否否半是是否是否是否是否半半否否半否半半否半否否是是是否半
FULL-1
半否否否半否半否否否半否否半半否否半半半是否否半是否是否否半是否否否否否是否否否半否否是半否否是否是
FULL-2
否否否否否否否是半否半半半半半否半否否是否是否否否否否是是是半否是否否是是否否否是否否否是否否否否是
FULL-3
是是半否否否是否是是否否半是是否否半否否是半否是否否否是否是否否否半否是否半否否否否否是半否否否否否
FULL-4
否否是半是半否否否否是否否否否半半否是否否是否否否是否否半否否是否半否否半否半是半半否否否是是否半否
"""

result_RAG = """RAG-0
半是是半否是是否否是是半是是否是是否否是是是是是否半半是是是半否是是是是是半是是半否是是是半是半是是
RAG-1
是否是是是是是是是是是是是否否是是是是半否是是半否是半否是半是是是是否是是是否半否是是否是半是是是半
RAG-2
是是是是否是是是是否是是是是是是是半是否是否否是是是是半是否否是是否否是是是半是是是是是半是是否是半
RAG-3
是是是否是是是是否是是是是否半是否是否否是是是否是是否半是是是是是是是是是是是是是半否是是是是是是否
RAG-4
是是是是否是否是是否是是否否否是半是是是是否是是是否是是是是否是是是是否是是是是否是半是否是是否是否"""

target_chars = ["是", "半", "否"]
counter_FULL = Counter(result_FULL)
counter_RAG = Counter(result_RAG)

print("Full Search")
for char in target_chars:
    target_freq = counter_FULL.get(char, 0)
    print(f"{char}': {target_freq}")

print("\nMixed Search")
for char in target_chars:
    target_freq = counter_RAG.get(char, 0)
    print(f"{char}: {target_freq}")

Full Search
是': 52
半': 50
否': 148

Mixed Search
是': 170
半': 27
否': 53


In [23]:
from IPython.display import display, Markdown, clear_output
import time

def gradually_MDdisplay(qa_pairs, delay=0.1):
    # 顯示所有問題和答案
    display_text = ""
    for question, answer in qa_pairs[:-1]:
        display_text += f"**Q: {question}**\n\n{answer}\n\n{'-'*70}\n\n\n\n"

    # 逐行想是最新的答案
    latest_question, latest_answer = qa_pairs[-1]
    display_text += f"**Q: {latest_question}**\n\n"
    clear_output(wait=True)
    display(Markdown(display_text))

    latest_answer_lines = latest_answer.split('\n')
    for line in latest_answer_lines:
        display_text += line + '\n'
        clear_output(wait=True)
        display(Markdown(display_text))
        time.sleep(delay)

    display_text += '\n' + '-'*70 + '\n\n\n\n'
    clear_output(wait=True)
    display(Markdown(display_text))

qa_pairs = []

while True:
    query = input("Q: ")
    if query.lower() == "再見!":
        print("掰掰~")
        break
    answer = query_rag_system(query)
    qa_pairs.append((query, answer))
    gradually_MDdisplay(qa_pairs, delay=0.3)

**Q: 哈囉**

哈囉，有什麼我可以幫忙的嗎？

----------------------------------------------------------------------



**Q: 推薦幾堂通識課給我**

1. **經典通識講座**：這門課每週有不同講者演講，內容豐富且多樣。沒有考試，只需寫心得報告，適合想輕鬆修課的同學。

2. **經典通識**：主要是聽演講並寫心得報告，教師給分甜，出席率重要，但作業可以用網路資料補足。

3. **溝通與談判**：英文授課，內容實用，有助提升溝通技能。考試有講義幫忙畫重點，報告也不難。

4. **英文小組討論**：強化英語口說能力，有多次討論和即席發言活動，適合想提高英語口語的同學。

----------------------------------------------------------------------



**Q: 比較幾位網路程式設計概論的教授們**

王協源教授的課程內容較為深入，特別強調系統層面的知識，如 OS 和網路程式設計，考試和作業難度較高，給分嚴格。袁賢銘教授的課程設計較靈活，作業多元化，但上課出席率不高，作業設計有些奇怪。袁賢明教授的課程使用投影片，遠距教學，作業設計較為完整，滿分容易突破。總體而言，王協源教授的課程內容豐富但挑戰性高，袁賢銘教授的課程設計較為鬆散，而袁賢明教授則注重實作和學生的參與。

----------------------------------------------------------------------



**Q: 資工系有哪些必修課程?**

資工系的必修課程包括：
1. 資訊工程研討
2. 機率
3. 資訊工程專題（一、二）
4. 線性代數
更多詳細課程需要查詢課程規劃。

----------------------------------------------------------------------



**Q: 彭文志教授有開過哪些課程?**

彭文志教授的課程包括《文學經典:當代中國文學選讀》。這門課主要探討當代中國文學作品。上課方式和考試作業詳情不明。課程可能注重出席率，建議不要遲到。

----------------------------------------------------------------------



**Q: 資料庫系統概論課主要在教什麼?**

資料庫系統概論課主要教以下內容：

1. 資料庫系統的基本概念及其優點。
2. ER 和 EER 模型，用於設計資料模型。
3. 關聯模型及其設計方法，包括正規化和函數相依性。
4. SQL 語法及關聯代數。
5. 資料庫的檔案結構和索引技術。
6. 交易處理、並發控制及資料恢復。
7. 查詢優化等資料庫系統背後的原理與架構。

----------------------------------------------------------------------



**Q: 有哪些課程是全英文授課?**

只有「英文會話」課是全英文授課。

----------------------------------------------------------------------



**Q: 推薦幾堂很涼的通識課給我**

1. **經典通識講座**：每週聽演講，作業輕鬆，給分甜。
2. **經典通識**：每週聽演講，有作業但內容輕鬆，給分高。
3. **溝通與談判**：英文授課，考試和報告輕鬆，課堂氣氛輕鬆。
4. **行動創新服務**：課堂內容隨興，作業和報告輕鬆，給分甜。

----------------------------------------------------------------------



**Q: 哪些課程的loading很重?**

1. 進階物件導向程式設計
2. 計算機網路管理

----------------------------------------------------------------------



**Q: 哪些課程會看出席率?**

「統計學發展史」和「統計方法與資料分析」這兩門課程會看出席率。

----------------------------------------------------------------------



**Q: 有沒有又涼又甜的課程?**

有，「臺灣史」這門課非常涼且甜。老師上課輕鬆，沒作業和考試，只需出席。期末報告可自由選擇要不要做，課程內容簡單像聽故事。教授張慧安人很好，會鼓勵吃早餐。

----------------------------------------------------------------------



**Q: 哪些教授有教過計算機結構?**

劉志尉和周志成兩位教授都有教過計算機結構。

----------------------------------------------------------------------



**Q: 線性代數課的考試有幾次?**

三次期中考和六次作業。

----------------------------------------------------------------------



**Q: 人工智慧概論的評分方式是?**

期中考和期末考合計佔60%，三次手寫作業佔10%，程式專案1（8-puzzle）佔12%，程式專案2（dala）佔13%，基本分5%，點名加分2%。總成績經過公式調整，最高可達99分。

----------------------------------------------------------------------



**Q: 掰掰**

掰掰，祝你一切順利！

----------------------------------------------------------------------



**Q: 再見**

再見，祝你一切順利。

----------------------------------------------------------------------





Q: 再見!
掰掰~
